In [21]:
import urllib.request
import json
import zlib
import re

def load_json_from(url):
    rawdata = urllib.request.urlopen(url).read()
    return json.loads(rawdata.decode('utf-8'))

def load_gziped_json_from(url):
    rawdata = zlib.decompress(urllib.request.urlopen(url).read(), 16+zlib.MAX_WBITS)
    return json.loads(rawdata.decode('utf-8'))

In [22]:
raw_tournaments = load_json_from('http://live.leanpoker.org/api/tournament')
tournament_ids = list(map(lambda t: t['id'], filter(lambda t: t['status'] == 'ended',raw_tournaments)))

In [23]:
def get_team_data_for(id):
    tournament = load_json_from('http://live.leanpoker.org/api/tournament/' + id)
    return sorted(tournament['teams'], key=lambda x: -x['points'])

In [24]:
def increment(dict, key, amount = 1):
    if key in dict:
        dict[key] += amount
    else:
        dict[key] = amount

### Fetch tournaments

In [5]:
tournaments = []
for tid in tournament_ids:
    print('.',end='')
    tournaments.append(get_team_data_for(tid))

.....................................................

### Build some stats

In [6]:
language_counts = {}
winning_language_counts = {}
for tdata in tournaments: 
    if 'language_id' in tdata[0]:
        increment(winning_language_counts, tdata[0]['language_id'])
        for t in tdata:
            increment(language_counts, t['language_id'])

In [7]:
winrate = {}
for lang_id in language_counts:
    if lang_id in winning_language_counts:
        winrate[lang_id] = winning_language_counts[lang_id] / language_counts[lang_id]
    else:
        winrate[lang_id] = 0
        winning_language_counts[lang_id] = 0

In [8]:
import functools
average_language_lift = {}
for tdata in tournaments: 
    if 'language_id' in tdata[0]:
        total_points = functools.reduce(lambda s, t: s+t, map(lambda t: t['points'],tdata))
        for team in tdata:
            lift = (team['points'] * len(tdata))/total_points-1
            increment(average_language_lift, team['language_id'], lift)
            
for lang_id in language_counts:
    average_language_lift[lang_id] /= language_counts[lang_id]

In [9]:
for lang_id in language_counts:
    print(','.join([lang_id, str(language_counts[lang_id]), str(winning_language_counts[lang_id]), str(winrate[lang_id]), str(average_language_lift[lang_id])]))

clojure,3,0,0,-0.013277327739145325
csharp,25,5,0.2,-0.10319082578412274
ts,1,1,1.0,0.13246681415929196
groovy,2,0,0,0.18969961834662308
cpp,6,1,0.16666666666666666,0.0034205835456073697
js,51,11,0.21568627450980393,0.00523852069650884
go,4,0,0,-0.35842439992782926
python,29,8,0.27586206896551724,0.051558928266389754
kotlin,1,0,0,-0.3036684782608695
elixir,2,0,0,0.1555397328694091
java,40,4,0.1,-0.020670558383870854
ruby,29,9,0.3103448275862069,0.05798345935850648
scala,6,2,0.3333333333333333,0.14729869746389015
haskell,4,2,0.5,-0.07064867201912906
php,9,2,0.2222222222222222,0.048334262338414945
perl,2,0,0,-0.0698888155569875


## Download and convert games

In [25]:
def get_games_for(id):
    return load_gziped_json_from('http://live.leanpoker.org/api/tournament/' + id + '/game')

In [26]:
def get_game_id_and_winner(game):
    winner = -1
    for i in range(len(game['teams'])):
        if game['teams'][i]['gained'] == 5:
            winner = i
            
    return { 
        'id': game['id'],
        'winner': winner
    }

def get_game_ids_and_winners_for(id):
    return list(map(get_game_id_and_winner, get_games_for(id)))

In [27]:
# http://live.leanpoker.org/api/tournament/58b538ab8835920004000021/game/58b5969480f87800040002b3/log
def get_game_log_for(tournament_id, game_id):
    return load_gziped_json_from('http://live.leanpoker.org/api/tournament/' + tournament_id + '/game/' + game_id + '/log')
    

In [28]:

game_log = get_game_log_for('58b538ab8835920004000021','58b5969480f87800040002b3')

In [47]:
def get_hole_cards(log_line):
    return list(map(lambda t: t['hole_cards'],log_line['game_state']['players']))

def get_bets(log_line):
    return list(map(lambda t: t['bet'], log_line['game_state']['players']))

def get_stacks(log_line):
    return list(map(lambda t: t['stack'], log_line['game_state']['players']))

def get_winner(original_stacks, current_stacks):
    for i in range(len(original_stacks)):
        if original_stacks[i] < current_stacks[i]:
            return i
    return -1

def parse_message(message):
    match = re.search('made a bet of (\d+) \((\w+)\)', message)
    if match: 
        return { 'bet': int(match.group(1)), 'type': match.group(2) }
    else:
        return { 'bet': 0, 'type': 'fold' }

def convert(game_log, game_winner):
    hands = []
    hand = {}
    bets = []
    for log_line in game_log:
        if 'type' in log_line:
            if log_line['type'] == 'showdown':
                hand['community_cards'] = log_line['game_state']['community_cards']
                hand['hole_cards'] = get_hole_cards(log_line)
                hand['blind'] = log_line['game_state']['big_blind']
                hand['dealer'] = log_line['game_state']['dealer']
                hand['bets'] = bets
                hand['initial_stacks'] = stacks
                hand['tournament_id'] = log_line['game_state']['tournament_id']
                hand['game_id'] = log_line['game_state']['game_id']
                hand['round'] = log_line['game_state']['round']

            elif log_line['type'] == 'bet':
                message_data = parse_message(log_line['message'])
                bets[-1].append({
                    'bets': get_bets(log_line), 
                    'on_turn': log_line['on_turn'],
                    'type': message_data['type'],
                    'bet': message_data['bet']
                })
            elif log_line['type'] == 'card_deal':
                if len(log_line['game_state']['community_cards']) in [3,4,5]:
                    bets.append([])
            elif log_line['type'] == 'winner_announcement' and not 'winner' in hand:
                hand['winner'] = get_winner(stacks, get_stacks(log_line))
        else:
            stacks = get_stacks(log_line)
            if 'bets' in hand: 
                hand['remaining_stacks'] = stacks
                hand['game_winner'] = game_winner
                hands.append(hand)
            hand = {}
            bets = [[]]
    return hands

In [48]:
convert(game_log, 5)

[{'bets': [[{'bet': 2, 'bets': [0, 0, 2, 0, 0], 'on_turn': 2, 'type': 'raise'},
    {'bet': 4, 'bets': [0, 0, 2, 4, 0], 'on_turn': 3, 'type': 'raise'},
    {'bet': 104, 'bets': [0, 0, 2, 4, 104], 'on_turn': 4, 'type': 'raise'},
    {'bet': 0, 'bets': [0, 0, 2, 4, 104], 'on_turn': 0, 'type': 'fold'},
    {'bet': 104, 'bets': [0, 104, 2, 4, 104], 'on_turn': 1, 'type': 'call'},
    {'bet': 102, 'bets': [0, 104, 104, 4, 104], 'on_turn': 2, 'type': 'call'},
    {'bet': 0, 'bets': [0, 104, 104, 4, 104], 'on_turn': 3, 'type': 'fold'}],
   [{'bet': 109, 'bets': [0, 104, 213, 4, 104], 'on_turn': 2, 'type': 'raise'},
    {'bet': 109, 'bets': [0, 104, 213, 4, 213], 'on_turn': 4, 'type': 'call'},
    {'bet': 109, 'bets': [0, 213, 213, 4, 213], 'on_turn': 1, 'type': 'call'}],
   [{'bet': 218, 'bets': [0, 213, 431, 4, 213], 'on_turn': 2, 'type': 'raise'},
    {'bet': 218, 'bets': [0, 213, 431, 4, 431], 'on_turn': 4, 'type': 'call'},
    {'bet': 218, 'bets': [0, 431, 431, 4, 431], 'on_turn': 1, 'type

In [49]:
def fetch_games_for(tournament_id):
    gids = get_game_ids_and_winners_for(tournament_id)
    for gid in gids:
        log = convert(get_game_log_for(tournament_id, gid['id']), gid['winner'])
        f = open('/data/poker/rawdata/' + tournament_id + '_' + gid['id'] + '.json', 'w')
        f.write(json.dumps(log))
        f.close
        print('.', end='')

In [50]:
fetch_games_for('58430f4e7e7a0f000403a6d5')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................